In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import os
import re
import glob

In [2]:
import configargparse

p = configargparse.ArgParser(description=__doc__)
p.add('-c', '--config_file', is_config_file=True, env_var='CONFIG_FILE', default='config.txt')
p.add('--external_data_dir', required=True)
args = p.parse_known_args()[0]

In [3]:
args.external_data_dir

'/Users/dstuck/projects/election/ext_data'

# Pull in Census Demographics

## 2011 5 Year Estimate

In [4]:
race_by_county = pd.read_csv(
    os.path.join(args.external_data_dir, 'ACS_5YR/ACS_11_5YR_B02001_with_ann.csv')
)

In [5]:
#race_by_county

## 2015 Estimates

In [6]:
'''
The key for the YEAR variable is as follows: 
1 = 4/1/2010 Census population
2 = 4/1/2010 population estimates base 
3 = 7/1/2010 population estimate
4 = 7/1/2011 population estimate 
5 = 7/1/2012 population estimate 
6 = 7/1/2013 population estimate 
7 = 7/1/2014 population estimate 
8 = 7/1/2015 population estimate

The key for AGEGRP is as follows: 
0 = Total
1 = Age 0 to 4 years 
2 = Age 5 to 9 years
3 = Age 10 to 14 years
4 = Age 15 to 19 years
5 = Age 20 to 24 years
6 = Age 25 to 29 years
7 = Age 30 to 34 years
8 = Age 35 to 39 years
9 = Age 40 to 44 years
10 = Age 45 to 49 years 
11 = Age 50 to 54 years 
12 = Age 55 to 59 years 
13 = Age 60 to 64 years 
14 = Age 65 to 69 years 
15 = Age 70 to 74 years 
16 = Age 75 to 79 years 
17 = Age 80 to 84 years 
18 = Age 85 years or older

TOT_POP - Total population
WA_MALE - White alone male population
WA_FEMALE
BAC_MALE - Black or African American alone or in combination male population
BAC_FEMALE
''';

In [7]:
#Download file info here: http://www.census.gov/popest/data/counties/asrh/2015/CC-EST2015-ALLDATA.html
census_df = pd.read_csv(
    'http://www.census.gov/popest/data/counties/asrh/2015/files/CC-EST2015-ALLDATA.csv'
)

In [8]:
census_df.columns = [col.lower() for col in census_df.columns]

In [9]:
demos_2015_voters_df = census_df.query('year == 8 and agegrp >= 5')

In [10]:
demos_2015_voters_df = demos_2015_voters_df.drop(['sumlev', 'year', 'agegrp'], axis=1).groupby(
    ['state', 'stname', 'county', 'ctyname']
).sum().reset_index()

In [11]:
demos_2015_voters_df.head()

,state,stname,county,ctyname,tot_pop,tot_male,tot_female,wa_male,wa_female,ba_male,...,hwac_male,hwac_female,hbac_male,hbac_female,hiac_male,hiac_female,haac_male,haac_female,hnac_male,hnac_female
0,1,Alabama,1,Autauga County,40372,19225,21147,15314,16399,3383,...,400,405,47,37,19,23,4,6,13,7
1,1,Alabama,3,Baldwin County,154649,74316,80333,65923,70973,6430,...,2865,2406,135,142,118,94,20,27,20,20
2,1,Alabama,5,Barbour County,20379,11057,9322,5738,4849,5079,...,313,196,45,30,47,20,0,2,14,9
3,1,Alabama,7,Bibb County,17438,9372,8066,6726,6519,2525,...,208,147,15,7,5,3,1,2,15,2
4,1,Alabama,9,Blount County,43050,21076,21974,20307,21207,387,...,1612,1163,45,25,24,48,5,5,16,13


In [12]:
def combine_st_ct(state, county):
    return '{}{:03d}'.format(state, county)

In [13]:
demos_2015_voters_df.loc[:, 'fips'] = demos_2015_voters_df.apply(
    lambda x: combine_st_ct(x['state'], x['county']), axis=1
)

In [14]:
demos_2015_voters_df = demos_2015_voters_df.set_index('fips')

In [15]:
#demos_2015_voters_df.tot_pop

In [16]:
def fraction_by_race(data, race='bac'):
    return (data[race+'_male'] + data[race+'_female']) / float(data['tot_pop'])

In [17]:
demos_2015_voters_df.loc[:, 'bac_frac'] = demos_2015_voters_df.apply(lambda x: fraction_by_race(x, race='bac'), axis=1)

In [18]:
demos_2015_voters_df.loc[:, 'wa_frac'] = demos_2015_voters_df.apply(lambda x: fraction_by_race(x, race='wa'), axis=1)

In [20]:
demos_2015_voters_df.loc[:, 'wac_frac'] = demos_2015_voters_df.apply(lambda x: fraction_by_race(x, race='wa'), axis=1)

In [21]:
demos_2015_voters_df.to_csv('../data/demos_2015.csv.gz', compression='gzip')

## Voter Rights Act Counties

In [19]:
# https://www.justice.gov/crt/jurisdictions-previously-covered-section-5

In [20]:
county_to_code_mapping = pd.read_csv(os.path.join(
        args.external_data_dir, 'county_election_results_2016/state_county_fips.csv'
    ))

In [49]:
vra_states = [
    'al', 'ak', 'az', 'ga', 'la', 'ms', 'sc', 'tx', 'va'
]


In [47]:
vra_counties = {
    'ca': [
        'kings',
        'monterey',
        'yuba',
    ],
    'fl': [
        'collier', 'hardee', 'hendry', 'hillsborough', 'monroe'
    ],
    'ny': ['bronx', 'bronx', 'kings', 'kings', 'new york'],
    'nc': [
        'anson',
        'beaufort',
        'bertie',
        'bladen',
        'camden',
        'caswell',
        'chowan',
        'cleveland',
        'craven',
        'cumberland',
        'edgecombe',
        'franklin',
        'gaston',
        'gates',
        'granville',
        'greene',
        'guilford',
        'halifax',
        'harnett',
        'hertford',
        'hoke',
        'jackson',
        'lee',
        'lenoir',
        'martin',
        'nash',
        'northampton',
        'onslow',
        'pasquotank',
        'perquimans',
        'person',
        'pitt',
        'robeson',
        'rockingham',
        'scotland',
        'union',
        'vance',
        'washington',
        'wayne',
        'wilson'
    ],
    'sd': ['shannon', 'todd'],
    'mi': ['allegan', 'saginaw']
}

In [70]:
vra_county_df = pd.DataFrame([])
i=0
for state, counties in vra_counties.iteritems():
    for c in counties:
        vra_county_df.loc[i, 'abbr_state'] = state
        vra_county_df.loc[i, 'county'] = c
        i+=1
        #print county_to_code_mapping.query('abbr_state=="{}" and county=="{}"'.format(state, c))

In [71]:
vra_state_df = pd.DataFrame(vra_states, columns=['abbr_state'])

In [76]:
full_vra_counties = pd.concat([
        vra_state_df.merge(county_to_code_mapping)[['fips']],
        vra_county_df.merge(county_to_code_mapping)[['fips']]
    ]).set_index('fips')

In [80]:
full_vra_counties.loc[:, 'vra_county'] = True

In [83]:
full_vra_counties = full_vra_counties.join(
    county_to_code_mapping.set_index('fips')[[]], how='outer'
).fillna('False')

## New Voter Restrictions

In [ ]:
# https://www.aclu.org/map/voter-suppression-laws-whats-new-2012-presidential-election

In [85]:
restricted_states = [
    'az', 'tx', 'ks', 'nd', 'wi', 'in', 'oh',
    'nh', 'ri', 'tn', 'nc', 'ms', 'al', 'ga'
]

In [91]:
restricted_states_df = pd.DataFrame(
    restricted_states, columns=['abbr_state']
).merge(county_to_code_mapping).set_index('fips')[[]]

In [93]:
restricted_states_df.loc[:, 'voter_restrictions'] = True

In [98]:
full_vra_counties = full_vra_counties.join(restricted_states_df).fillna(False)

In [109]:
full_vra_counties = full_vra_counties.reset_index().drop_duplicates().set_index('fips')

In [110]:
full_vra_counties.to_csv('../data/vra_counties_2015.csv.gz', compression='gzip')

# Load Election Results

## 2016 Results

In [341]:
pres_2016_df = pd.read_csv(os.path.join(
        args.external_data_dir, 'county_election_results_2016/county_election_results_2016.csv'
    ))

In [342]:
#Convert strings, NANs, and '-'s to floats-
pres_2016_df.loc[:, 'votes'] = pres_2016_df.votes.fillna('NAN').apply(lambda x: float(x.replace(',', '').replace('-', 'NAN')))

In [344]:
pres_2016_df = pres_2016_df.dropna(axis=0, subset=['fips'])
pres_2016_df.loc[:, 'fips'] = pres_2016_df.fips.astype(int)
#pres_2016_df = pres_2016_df.set_index('fips')

In [345]:
digit_cleaner = re.compile('(\d+[\.\d]?)')
def clean_prefixed_digits(dstr):
    match = re.search(digit_cleaner, str(dstr))
    if match:
        return float(match.groups()[0])
    else:
        return np.nan

In [346]:
pres_2016_df.loc[:, 'percent_complete'] = pres_2016_df.percent_complete.apply(clean_prefixed_digits)

In [347]:
#pres_2016_df[pres_2016_df.percent_complete<100].abbr_state.value_counts()/4

In [348]:
party_mapping_2016 = {
    'Hillary Clinton':'D',
    'Gary Johnson':'O',
    'Jill Stein':'O',
    'Donald Trump':'R',
    'Other':'O',
    'Evan McMullin':'O'
}

In [349]:
pres_2016_df.loc[:, 'party'] = pres_2016_df.candidate.apply(lambda x: party_mapping_2016.get(x))

In [350]:
pres_2016_df.loc[pres_2016_df.percent_complete<99, 'votes'] = np.nan

In [351]:
pres_2016_df = pres_2016_df.groupby(['fips', 'party'])[['votes']].sum()

In [361]:
pres_2016_df.loc[:, 'percent_won'] = pres_2016_df.votes / pres_2016_df.groupby(level=0).votes.sum()

In [362]:
pres_2016_df.head()

votes  percent_won
fips party                      
1001 D       5908.0     0.239569
     O        643.0     0.026074
     R      18110.0     0.734358
1003 D      18409.0     0.195653
     O       2901.0     0.030832

In [363]:
pres_2016_df.to_csv('../data/cleaned_pres_2016_df.csv.gz', compression='gzip')

In [360]:
#pd.read_csv('../data/cleaned_pres_2016_df.csv.gz', compression='gzip', index_col=['fips', 'party'])

## 2012 Results

In [312]:
state_list = []
for fname in glob.glob(os.path.join(
    args.external_data_dir, 'county_election_results_2012/data/*'
)):
    state_df = pd.read_csv(fname)
    state_df.loc[:, 'abbr_state'] = os.path.splitext(os.path.basename(fname))[0]
    state_list.append(state_df)
pres_2012_df = pd.concat(state_list)

In [313]:
pres_2012_df = pres_2012_df.dropna(subset=['fips'])
pres_2012_df.loc[:, 'fips'] = pres_2012_df.fips.astype(int)
#pres_2012_df = pres_2012_df.set_index('fips')

In [314]:
def party_mapping_2012(cand_str):
    if 'obama' in cand_str.lower():
        return 'D'
    elif 'romney' in cand_str.lower():
        return 'R'
    else:
        return 'O'

In [315]:
pres_2012_df.loc[:, 'party'] = pres_2012_df.candidate.apply(party_mapping_2012)

In [316]:
pres_2012_df.head()

,abbr_state,candidate,county,fips,precinct,town,votes,party
0,al,BARACK OBAMA / JOE BIDEN (D),Autauga,1001,NaN,NaN,6363,D
1,al,MITT ROMNEY /PAUL RYAN (R),Autauga,1001,NaN,NaN,17379,R
2,al,"VIRGIL H. GOODE, JR. / JAMES CLYMER (I)",Autauga,1001,NaN,NaN,31,O
3,al,GARY JOHNSON / JIM GRAY (I),Autauga,1001,NaN,NaN,137,O
4,al,JILL STEIN / CHERI HONKALA (I),Autauga,1001,NaN,NaN,22,O


In [317]:
## Double check mapping
# pres_2012_df.query('party=="D"').groupby(['candidate', 'party']).county.count().sort_values(ascending=False)
# pres_2012_df.query('party=="R"').groupby(['candidate', 'party']).county.count().sort_values(ascending=False)
# list(
#     pres_2012_df.query('party=="O"').groupby(
#         ['candidate', 'party']
#     ).votes.sum().sort_values(ascending=False).reset_index().candidate
# )

In [318]:
pres_2012_df = pres_2012_df[['fips', 'votes', 'party']]

In [322]:
pres_2012_df = pres_2012_df.groupby(['fips', 'party'])[['votes']].sum()

In [325]:
pres_2012_df.loc[:, 'percent_won'] = pres_2012_df.votes / pres_2012_df.groupby(level=0).votes.sum()

In [326]:
pres_2012_df.head()

votes  percent_won
fips party                    
1001 D       6363     0.265424
     O        231     0.009636
     R      17379     0.724941
1003 D      18424     0.215508
     O       1051     0.012294

In [364]:
pres_2012_df.to_csv('../data/cleaned_pres_2012_df.csv.gz', compression='gzip')